### Импорты, параметры ColBERT

In [1]:
from interface import prepare_tsv, save_index, top_n_similar, Collection
import pandas as pd
import os

ckpt_pth = "/home/sondors/Documents/ColBERT_weights/bert-base-multilingual-cased_dim_768_bsize_230_lr04_use_ib_negatives/none/2024-01/27/16.55.29/checkpoints/colbert-2998-finish"
experiment = "bert-base-multilingual-cased-2998"

doc_maxlen = 300
nbits = 2   # bits определяет количество битов у каждого измерения в семантическом пространстве во время индексации
nranks = 1  # nranks определяет количество GPU для использования, если они доступны
kmeans_niters = 4 # kmeans_niters указывает количество итераций k-means кластеризации; 4 — хороший и быстрый вариант по умолчанию.  

### Подготовка моделей (на что матчить оффера - это может быть и смесь офферов и моделей)

In [2]:
pth_models = "/home/sondors/Documents/price/ColBERT_data/18_categories/test/models_18_categories.csv"
df_models = pd.read_csv(pth_models, sep=";")
df_models

/tmp/ipykernel_31058/17807022.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_models = pd.read_csv(pth_models, sep=";")


,model_id,average_price,name,brand_name,full_name,comment,category_name,category_id
0,623742,NaN,920-005619,Logitech,Logitech 920-005619,NaN,"чехлы, обложки для гаджетов (телефонов, планше...",3994
1,721952,NaN,Zipper Bag,Hama,Hama Zipper Bag,NaN,"чехлы, обложки для гаджетов (телефонов, планше...",3994
2,721970,NaN,CC-3064,Nokia,Nokia CC-3064,NaN,"чехлы, обложки для гаджетов (телефонов, планше...",3994
3,751488,990.0,CKS-X7/R,Sony,Sony CKS-X7/R,NaN,"чехлы, обложки для гаджетов (телефонов, планше...",3994
4,751989,NaN,EP-031023,Era Pro,Era Pro EP-031023,NaN,"чехлы, обложки для гаджетов (телефонов, планше...",3994
...,...,...,...,...,...,...,...,...
103209,7049424,16459.0,MD-108,Mivo,Mivo MD-108,NaN,портативная акустика,3904
103210,7049425,8812.0,MD-165,Mivo,Mivo MD-165,NaN,портативная акустика,3904
103211,7049426,4240.0,Boost 20W,Rocket,Rocket Boost 20W,NaN,портативная акустика,3904
103212,7049427,2990.0,Motion 10W,Rocket,Rocket Motion 10W,NaN,портативная акустика,3904


In [3]:
id_category = {
    # 3902: 'диктофоны, портативные рекордеры',
    # 510402: 'электронные книги',
    # 4302: 'автомобильные телевизоры, мониторы',
    # 2815: 'смарт-часы и браслеты',
    # 3901: 'портативные медиаплееры',
    # 3904: 'портативная акустика',
    # 2801: 'мобильные телефоны',
    # 3908: 'VR-гарнитуры (VR-очки, шлемы, очки виртуальной реальности, FPV очки для квадрокоптеров)',
    510401: 'планшетные компьютеры и мини-планшеты',
    # 2102: 'наушники, гарнитуры, наушники c микрофоном',
    # 3903: 'радиоприемники, радиобудильники, радиочасы',
    # 3907: 'магнитолы',
    # 280801: 'GPS-навигаторы'
    }

dst_fld = "/home/sondors/Documents/price/ColBERT/tutorial"
for cat_id in id_category.keys():
    category_models = df_models[df_models.category_id == cat_id].reset_index(drop=True)
    prepare_tsv(category_models, dst_fld, cat_id)

### Индексируем модели и сохраняем индекс на диск

In [4]:
for cat_id in id_category.keys():
    models_colbert = Collection(path=os.path.join(dst_fld, "tsv", f"{cat_id}_models.tsv"))
    index_name = f'models_{cat_id}_{nbits}bits'
    indexer = save_index(ckpt_pth, doc_maxlen, nbits, kmeans_niters, nranks, dst_fld, experiment, models_colbert, index_name)

[Feb 07, 19:42:47] #> Loading collection...
0M 


[Feb 07, 19:42:47] #> Note: Output directory /home/sondors/Documents/price/ColBERT/tutorial/bert-base-multilingual-cased-2998/indexes/models_510401_2bits already exists


[Feb 07, 19:42:47] #> Will delete 10 files already at /home/sondors/Documents/price/ColBERT/tutorial/bert-base-multilingual-cased-2998/indexes/models_510401_2bits in 20 seconds...
#> Starting...
{
    "query_token_id": "[unused0]",
    "doc_token_id": "[unused1]",
    "query_token": "[Q]",
    "doc_token": "[D]",
    "ncells": null,
    "centroid_score_threshold": null,
    "ndocs": null,
    "load_index_with_mmap": false,
    "index_path": null,
    "nbits": 2,
    "kmeans_niters": 4,
    "resume": false,
    "similarity": "cosine",
    "bsize": 64,
    "accumsteps": 1,
    "lr": 0.0001,
    "maxsteps": 2998,
    "save_every": null,
    "warmup": 0,
    "warmup_bert": null,
    "relu": false,
    "nway": 2,
    "use_ib_negatives": true,
    "reranker": false,
    "dis

/home/sondors/anaconda3/envs/colbert_cpu/lib/python3.8/site-packages/torch/cuda/amp/grad_scaler.py:118: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")
  0%|          | 0/50 [00:00<?, ?it/s]/home/sondors/anaconda3/envs/colbert_cpu/lib/python3.8/site-packages/torch/amp/autocast_mode.py:202: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


[Feb 07, 19:43:10] [0] 		 # of sampled PIDs = 3839 	 sampled_pids[:3] = [1706, 3001, 41]
[Feb 07, 19:43:10] [0] 		 #> Encoding 3839 passages..


100%|██████████| 10/10 [00:06<00:00,  1.44it/s]
WARNING clustering 38214 points to 2048 centroids: please provide at least 79872 training points


[Feb 07, 19:43:47] [0] 		 avg_doclen_est = 10.477989196777344 	 len(local_sample) = 3,839
[Feb 07, 19:43:47] [0] 		 Creaing 2,048 partitions.
[Feb 07, 19:43:47] [0] 		 *Estimated* 40,225 embeddings.
[Feb 07, 19:43:47] [0] 		 #> Saving the indexing plan to /home/sondors/Documents/price/ColBERT/tutorial/bert-base-multilingual-cased-2998/indexes/models_510401_2bits/plan.json ..
Clustering 38214 points in 768D to 2048 clusters, redo 1 times, 4 iterations
  Preprocessing in 0.02 s
  Iteration 3 (1.13 s, search 1.10 s): objective=8413.87 imbalance=1.475 nsplit=0       
[0.014, 0.013, 0.015, 0.013, 0.017, 0.013, 0.014, 0.014, 0.015, 0.015, 0.013, 0.013, 0.013, 0.013, 0.013, 0.014, 0.013, 0.014, 0.014, 0.015, 0.014, 0.016, 0.013, 0.014, 0.014, 0.016, 0.013, 0.013, 0.012, 0.015, 0.015, 0.015, 0.013, 0.014, 0.015, 0.012, 0.012, 0.013, 0.013, 0.012, 0.016, 0.015, 0.015, 0.014, 0.016, 0.012, 0.013, 0.012, 0.013, 0.012, 0.014, 0.012, 0.016, 0.014, 0.013, 0.014, 0.014, 0.016, 0.012, 0.012, 0.014, 0.

0it [00:00, ?it/s]
100%|██████████| 50/50 [00:31<00:00,  1.57it/s]

100%|██████████| 10/10 [00:07<00:00,  1.34it/s]


[Feb 07, 19:44:28] [0] 		 #> Saving chunk 0: 	 3,839 passages and 40,225 embeddings. From #0 onward.


1it [00:40, 40.19s/it]
100%|██████████| 2048/2048 [00:00<00:00, 137732.05it/s]


[Feb 07, 19:44:29] [0] 		 #> Checking all files were saved...
[Feb 07, 19:44:29] [0] 		 Found all files!
[Feb 07, 19:44:29] [0] 		 #> Building IVF...
[Feb 07, 19:44:29] [0] 		 #> Loading codes...
[Feb 07, 19:44:29] [0] 		 Sorting codes...
[Feb 07, 19:44:29] [0] 		 Getting unique codes...
[Feb 07, 19:44:29] #> Optimizing IVF to store map from centroids to list of pids..
[Feb 07, 19:44:29] #> Building the emb2pid mapping..
[Feb 07, 19:44:29] len(emb2pid) = 40225
[Feb 07, 19:44:29] #> Saved optimized IVF to /home/sondors/Documents/price/ColBERT/tutorial/bert-base-multilingual-cased-2998/indexes/models_510401_2bits/ivf.pid.pt
[Feb 07, 19:44:29] [0] 		 #> Saving the indexing metadata to /home/sondors/Documents/price/ColBERT/tutorial/bert-base-multilingual-cased-2998/indexes/models_510401_2bits/metadata.json ..
#> Joined...


### Поиск матча по индексу

In [5]:
offers = {
    0: 'Samsung Планшет Samsung Galaxy Tab S8, 8 ГБ/128 ГБ, Wi-Fi + Cellular, со стилусом, графит (Global)',
    1: 'Планшет Samsung Galaxy Tab S8 128GB 5G Silver (SM-X706B)',
    2: 'Планшет Samsung Galaxy Tab S8+ 128GB Wi-Fi Pink Gold (SM-X800)'
    }

src_fld = "/home/sondors/Documents/price/ColBERT/tutorial"
cat_id = 2801 # мобильные телефоны
cat_id = 510401 # планшеты
index_name = f'models_{cat_id}_{nbits}bits'
models_id_colbert = Collection(path=os.path.join(src_fld, "tsv", f"{cat_id}_models_id.tsv"))

top_n = top_n_similar(offers, src_fld, nranks, experiment, index_name, models_id_colbert, 5)

[Feb 07, 19:44:30] #> Loading collection...
0M 
[Feb 07, 19:44:30] Loading segmented_maxsim_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[Feb 07, 19:44:31] #> Loading codec...
[Feb 07, 19:44:31] #> Loading IVF...
[Feb 07, 19:44:31] Loading segmented_lookup_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


/home/sondors/anaconda3/envs/colbert_cpu/lib/python3.8/site-packages/torch/cuda/amp/grad_scaler.py:118: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


[Feb 07, 19:44:31] #> Loading doclens...


100%|██████████| 1/1 [00:00<00:00, 1810.23it/s]

[Feb 07, 19:44:31] #> Loading codes and residuals...


...

[Feb 07, 19:44:31] Loading filter_pids_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


[Feb 07, 19:44:32] Loading decompress_residuals_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


/home/sondors/anaconda3/envs/colbert_cpu/lib/python3.8/site-packages/torch/amp/autocast_mode.py:202: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')



#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . Samsung Планшет Samsung Galaxy Tab S8, 8 ГБ/128 ГБ, Wi-Fi + Cellular, со стилусом, графит (Global), 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([   101,    100,  37077,    524,  19079, 105694,  37077,  29723,  14248,
         10457,    156,  11396,    117,    129,    512,  18683,    120,  16196,
           512,  18683,    117,  52742,    118,  36448,    116,  29494,  18062,
           117,  10956,  67459,  19954,    102])
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])



3it [00:00, 298.07it/s]


### Получаем выдачу топ N моделей для каждого оффера

In [6]:
for i in range(len(top_n)):
    print(offers[i])
    for j in range(len(top_n[i]['model_ids'])):
        id = top_n[i]['model_ids'][j]
        sim = top_n[i]['similarity'][j]
        model = list(df_models[df_models.model_id == id]['full_name'])[0]
        print(f"\t{id}: {model} --> {round(float(sim), 2)}")
    print("_"*60)

Samsung Планшет Samsung Galaxy Tab S8, 8 ГБ/128 ГБ, Wi-Fi + Cellular, со стилусом, графит (Global)
	5144478: Samsung Galaxy Tab S8 --> 28.97
	5144477: Samsung Galaxy Tab S8+ --> 24.97
	937550: Samsung Galaxy Tab S2 9.7 SM-T813 --> 24.0
	5144479: Samsung Galaxy Tab S8 Ultra --> 23.98
	816746: Samsung Galaxy Tab S2 9.7 SM-T810 --> 23.64
____________________________________________________________
Планшет Samsung Galaxy Tab S8 128GB 5G Silver (SM-X706B)
	5144478: Samsung Galaxy Tab S8 --> 28.15
	5144477: Samsung Galaxy Tab S8+ --> 23.94
	937550: Samsung Galaxy Tab S2 9.7 SM-T813 --> 23.21
	623216: Samsung Galaxy Tab S 8.4 SM-T705 --> 22.82
	816746: Samsung Galaxy Tab S2 9.7 SM-T810 --> 22.65
____________________________________________________________
Планшет Samsung Galaxy Tab S8+ 128GB Wi-Fi Pink Gold (SM-X800)
	5144477: Samsung Galaxy Tab S8+ --> 26.03
	5144478: Samsung Galaxy Tab S8 --> 20.32
	4509798: Samsung Galaxy Tab S7+ 12.4 128Gb --> 18.63
	937550: Samsung Galaxy Tab S2 9.7 SM-T

In [7]:
top_n

[{'model_ids': [5144478, 5144477, 937550, 5144479, 816746],
  'similarity': [28.973285675048828,
   24.965259552001953,
   23.999181747436523,
   23.97533416748047,
   23.638072967529297]},
 {'model_ids': [5144478, 5144477, 937550, 623216, 816746],
  'similarity': [28.152374267578125,
   23.93535041809082,
   23.20758819580078,
   22.81966781616211,
   22.648759841918945]},
 {'model_ids': [5144477, 5144478, 4509798, 937550, 5144479],
  'similarity': [26.02865982055664,
   20.31949234008789,
   18.63184928894043,
   18.34659194946289,
   18.30866813659668]}]